In [1]:
import numpy as np
import pandas as pd
import re
#import tabula
import pdfplumber
from collections import namedtuple
import os
from datetime import date

In [ ]:
file_name = input("Por favor, cole aqui o nome do arquivo PDF que deseja trabalhar :")
file = file_name + ".pdf"

In [5]:
sub_te_re = re.compile(r'(SUBSIDIO TARIFARIO TE) (UN) (.*)')
sub_re = re.compile(r'(SUBSIDIO) (.*)')
sub_tusd_re = re.compile(r'(SUBSIDIO TARIFARIO TUSD) (UN) (.*)')
sub_taf_liq = re.compile(r'(SUBSIDIO TARIFARIO LIQUIDO) (UN) (.*)')

ener_con_re = re.compile(r'(ENERGIA ELET CONSUMO) (kWh) (.*) (CPIOSFINS) (.*) (.*) (.*)')

ener_re = re.compile (r'(ENERGIA ELET) (.*)')

ener_uso_re = re.compile (r'(ENERGIA ELET USO SISTEMA) (kWh) (.*)')

ener_con_re = re.compile (r'(ENERGIA ELET CONSUMO) (kWh) (.*)')

ener_te_res_re = re.compile (r'(ENERGIA ELET TE PER RESERV) (kWh) (.*)')
ener_usd_res_re = re.compile (r'(ENERGIA ELET USD PER RESERV) (kWh) (.*)')

ener_inj_te_re = re.compile (r'(ENERGIA INJETADA TE) (\d{2}/\d{4}) (kWh) (.*)')
ener_inj_tusd_re = re.compile (r'(ENERGIA INJETADA TUSD) (\d{2}/\d{4}) (kWh) (.*)')

ener_inj_rs_te_re = re.compile (r'(ENERGIA INJETADA RS TE) (\d{2}/\d{4}) (kWh) (.*)')
ener_inj_rs_tusd_re = re.compile (r'(ENERGIA INJETADA RS TUSD) (\d{2}/\d{4}) (kWh) (.*)')

un_re = re.compile(r'(Endereço:) (.*) (.*) (.*)')

dsc_re = re.compile(r'(DESC PROG TRN LEI EST 19812.19) (UN) (.*)')

ener_inj_ouc_mpt_te_re = re.compile(r'(ENERGIA INJ. OUC MPT TE) (\d{2}/\d{4}) (kWh) (.*)')
ENERGIA_INJ_OUC_MPT_TUSD_re = re.compile(r'(ENERGIA INJ. OUC MPT TUSD) (\d{2}/\d{4}) (kWh) (.*)')

ENERGIA_INJ_MUC_MPT_TE = re.compile(r'(ENERGIA INJ. MUC MPT TE) (\d{2}/\d{4}) (kWh) (.*)')
ENERGIA_INJ_MUC_MPT_TUSD = re.compile(r'(ENERGIA INJ. MUC MPT TUSD) (\d{2}/\d{4}) (kWh) (.*)') 

energia_inj_rs_muc_TE = re.compile(r'(ENERGIA INJ. RS MUC MPT TE) (\d{2}/\d{4}) (kWh) (.*)')
energia_inj_rs_muc_TUSD = re.compile(r'(ENERGIA INJ. RS MUC MPT TUSD) (\d{2}/\d{4}) (kWh) (.*)')

energia_inj_rs_TE = re.compile(r'(ENERGIA INJETADA RS TE) (\d{2}/\d{4}) (kWh) (.*)')
energia_inj_rs_TUSD = re.compile(r'(ENERGIA INJETADA RS TUSD) (\d{2}/\d{4}) (kWh) (.*)')


ENERGIA_INJ_RS_OUC_MPT_TUSD = re.compile(r'(ENERGIA INJ. RS OUC MPT TUSD) (\d{2}/\d{4}) (kWh) (.*)')
ENERGIA_INJ_RS_OUC_MPT_TE = re.compile(r'(ENERGIA INJ. RS OUC MPT TE) (\d{2}/\d{4}) (kWh) (.*)')

ENERGIA_INJ_RS_OUC_OPT_TE = re.compile(r'(ENERGIA INJ. RS OUC OPT TE) (\d{2}/\d{4}) (kWh) (.*)')
ENERGIA_INJ_RS_OUC_OPT_TUSD = re.compile(r'(ENERGIA INJ. RS OUC OPT TUSD) (\d{2}/\d{4}) (kWh) (.*)')







In [6]:
Line = namedtuple('Line', 'UC Itens_de_Fatura Valores')

In [7]:
lines = []
with pdfplumber.open(file) as pdf:
    pages = pdf.pages
    for page in pages:
        text = page.extract_text()
        for row in text.split('\n'):
            if un_re.findall(row):
                uc = un_re.search(row).group(4)
                #lines.append(Line(uc,itens_fat, nums))
                #rows.append(Line(itens_fat, nums.split()))
            
            elif ener_uso_re.search(row):
                itens_fat, nums = ener_uso_re.search(row).group(1), ener_uso_re.search(row).group(3)
                lines.append(Line(uc,itens_fat, nums))
                #rows.append(Line(en_pt_tusd_name, en_pt_tusd_val.split()))
            
            elif ener_con_re.search(row):
                itens_fat, nums = ener_con_re.search(row).group(1), ener_con_re.search(row).group(3)
                lines.append(Line(uc,itens_fat, nums))
            elif ener_te_res_re.search(row):
                itens_fat, nums = ener_te_res_re.search(row).group(1), ener_te_res_re.search(row).group(3)
                lines.append(Line(uc,itens_fat, nums))
            
            elif ener_usd_res_re.search(row):
                itens_fat, nums = ener_usd_res_re.search(row).group(1), ener_usd_res_re.search(row).group(3)
                lines.append(Line(uc,itens_fat, nums))
            
            elif ener_inj_te_re.search(row):
                itens_fat, nums = ener_inj_te_re.search(row).group(1), ener_inj_te_re.search(row).group(4)
                lines.append(Line(uc,itens_fat, nums))
            
            elif ener_inj_tusd_re.search(row):
                itens_fat, nums = ener_inj_tusd_re.search(row).group(1), ener_inj_tusd_re.search(row).group(4)
                lines.append(Line(uc,itens_fat, nums))
            
            elif ener_inj_rs_te_re.search(row):
                itens_fat, nums = ener_inj_rs_te_re.search(row).group(1), ener_inj_rs_te_re.search(row).group(4)
                lines.append(Line(uc,itens_fat, nums))
            
            elif ener_inj_rs_tusd_re.search(row):
                itens_fat, nums = ener_inj_rs_tusd_re.search(row).group(1), ener_inj_rs_tusd_re.search(row).group(4)
                lines.append(Line(uc,itens_fat, nums))
            
            elif sub_te_re.search(row):
                itens_fat, nums = sub_te_re.search(row).group(1), sub_te_re.search(row).group(3)
                lines.append(Line(uc,itens_fat, nums))
            
            elif sub_tusd_re.search(row):
                itens_fat, nums = sub_tusd_re.search(row).group(1), sub_tusd_re.search(row).group(3)
                lines.append(Line(uc,itens_fat, nums))
            
            elif sub_taf_liq.search(row):
                itens_fat, nums = sub_taf_liq.search(row).group(1), sub_taf_liq.search(row).group(3)
                lines.append(Line(uc,itens_fat, nums))
            
            elif dsc_re.search(row):
                itens_fat, nums = dsc_re.search(row).group(1), dsc_re.search(row).group(3)
                lines.append(Line(uc,itens_fat, nums))
            
            elif ener_inj_ouc_mpt_te_re.search(row):
                itens_fat, nums = ener_inj_ouc_mpt_te_re.search(row).group(1), ener_inj_ouc_mpt_te_re.search(row).group(4)
                lines.append(Line(uc,itens_fat, nums))
            
            elif ENERGIA_INJ_OUC_MPT_TUSD_re.search(row):
                itens_fat, nums = ENERGIA_INJ_OUC_MPT_TUSD_re.search(row).group(1), ENERGIA_INJ_OUC_MPT_TUSD_re.search(row).group(4)
                lines.append(Line(uc,itens_fat, nums))
            
            elif ENERGIA_INJ_MUC_MPT_TE.search(row):
                itens_fat, nums = ENERGIA_INJ_MUC_MPT_TE.search(row).group(1), ENERGIA_INJ_MUC_MPT_TE.search(row).group(4)
                lines.append(Line(uc,itens_fat, nums))
            
            elif ENERGIA_INJ_MUC_MPT_TUSD.search(row):
                itens_fat, nums = ENERGIA_INJ_MUC_MPT_TUSD.search(row).group(1), ENERGIA_INJ_MUC_MPT_TUSD.search(row).group(4)
                lines.append(Line(uc,itens_fat, nums))
            
            elif energia_inj_rs_muc_TE.search(row):
                itens_fat, nums = energia_inj_rs_muc_TE.search(row).group(1), energia_inj_rs_muc_TE.search(row).group(4)
                lines.append(Line(uc,itens_fat, nums))
            
            elif energia_inj_rs_muc_TUSD.search(row):
                itens_fat, nums = energia_inj_rs_muc_TUSD.search(row).group(1), energia_inj_rs_muc_TUSD.search(row).group(4)
                lines.append(Line(uc,itens_fat, nums))
            
            elif energia_inj_rs_TE.search(row):
                itens_fat, nums = energia_inj_rs_TE.search(row).group(1), energia_inj_rs_TE.search(row).group(4)
                lines.append(Line(uc,itens_fat, nums))
            
            elif energia_inj_rs_TUSD.search(row):
                itens_fat, nums = energia_inj_rs_TUSD.search(row).group(1), energia_inj_rs_TUSD.search(row).group(4)
                lines.append(Line(uc,itens_fat, nums))    
            
            elif ENERGIA_INJ_RS_OUC_MPT_TUSD.search(row):
                itens_fat, nums = ENERGIA_INJ_RS_OUC_MPT_TUSD.search(row).group(1), ENERGIA_INJ_RS_OUC_MPT_TUSD.search(row).group(4)
                lines.append(Line(uc,itens_fat, nums)) 
           
            elif ENERGIA_INJ_RS_OUC_MPT_TE.search(row):
                itens_fat, nums = ENERGIA_INJ_RS_OUC_MPT_TE.search(row).group(1), ENERGIA_INJ_RS_OUC_MPT_TE.search(row).group(4)
                lines.append(Line(uc,itens_fat, nums)) 
            
            elif ENERGIA_INJ_RS_OUC_OPT_TE.search(row):
                itens_fat, nums = ENERGIA_INJ_RS_OUC_OPT_TE.search(row).group(1), ENERGIA_INJ_RS_OUC_OPT_TE.search(row).group(4)
                lines.append(Line(uc,itens_fat, nums))
            
            elif ENERGIA_INJ_RS_OUC_OPT_TUSD.search(row):
                itens_fat, nums = ENERGIA_INJ_RS_OUC_OPT_TUSD.search(row).group(1), ENERGIA_INJ_RS_OUC_OPT_TUSD.search(row).group(4)
                lines.append(Line(uc,itens_fat, nums))
            

In [ ]:
dfl = pd.DataFrame(lines)
#display(dfl)

In [9]:
occurrence = dfl['Itens_de_Fatura'].value_counts().rename_axis('Items de Fatura ').reset_index(name='Ocorrência')

df_ocr = pd.DataFrame(occurrence)
#display(df_ocr)

In [10]:
uc_occurrence = dfl['UC'].value_counts().rename_axis('UC').reset_index(name='Ocorrência')

df_uc_ocr = pd.DataFrame(uc_occurrence)
#display(df_uc_ocr)

In [11]:
today = date.today()
today_string = str(today)

In [12]:
dfl.to_csv(os.path.join('directory_path',file_name + ' - Parsed - ' + today_string),index=False)

In [13]:
df_uc_ocr.to_csv(os.path.join('directory_path', file_name + 'Ocorrência das UCs - '  + today_string),index=False)

In [14]:
df_ocr.to_csv(os.path.join('directory_path',file_name + 'Ocorrência dos Items de Fatura - ' + today_string),index=False)